In [3]:
cd ..

/home/is/yuka-ot/dev


/home/is/yuka-ot/dev/dicomplete/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import pandas as pd
import torch
from src.generate import CustomDataset, Trainer, GenerateText
from transformers import T5Tokenizer, T5ForConditionalGeneration

/home/is/yuka-ot/dev/dicomplete/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
df = pd.read_csv('dicomplete/data/inputs/DISEASE_SIP-3_v202401_2.4_small.csv')

In [14]:
df

,ID,出現形,出現形_flag,出現形よみ,出現形よみ_flag,正規形,正規形_flag,正規形よみ,正規形よみ_flag,TREE,...,TYPE:症状,TYPE:症状_flag,TYPE:診断名,TYPE:診断名_flag,TYPE:否定,TYPE:否定_flag,FREQ:GLOBAL,FREQ:LOCAL,SOURCE,temp_flag
0,fe55650df39660acbe398b99908725d6,副作用,NaN,ふくさよう,C,-1,C,-1,B,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,15232.0,NaN,JST;JAMSMDIC,S
1,17eced89ff5b50b688b319d6c59016c3,合併症,NaN,がっぺいしょう,B,-1,B,-1,B,-1,...,0.0,C,0.0,C,NaN,NaN,14996.0,95-100%,MANBYO;JST;JAMSMDIC,S
2,b41877e5d586b1b557850fd91ad9fe5e,糖尿病,NaN,とうにょうびょう,S,糖尿病,S,とうにょうびょう,S,E14,...,0.0,C,1.0,C,NaN,NaN,10535.0,95-100%,MANBYO;JST;JAMSMDIC;Lilak,S
3,49aec2159aee4caa79922620af8dd5ef,再発,NaN,さいはつ,C,-1,C,-1,B,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,9771.0,NaN,JST;JAMSMDIC,S
4,4a5301964900637efac1f17211ac59a2,癌,NaN,がん,S,癌,S,がん,S,C809,...,0.0,C,1.0,C,NaN,NaN,6357.0,出現なし,MANBYO;JST;JAMSMDIC;Lilak,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,97ed72a3b0cf772e554551ab46dc50f9,おう吐，下痢,NaN,おうとげり,C,[ERR],NaN,NaN,NaN,A099,...,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,JST,S
9995,ba83573a17d7ede9319631c539c8541f,肝・腎障害,NaN,かんじんしょうがい,C,-1,C,-1,C,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,JST,S
9996,06224ae8cc755c8f117102df86fc43c9,転移性前立腺癌,NaN,てんいせいぜんりつせんがん,C,転移性前立腺癌,C,てんいせいぜんりつせんがん,C,C798,...,0.0,C,0.0,C,NaN,NaN,22.0,0-5%,MANBYO;JST,S
9997,c6c0d641086e5bbdf35f1574910598c6,ＭＡＳ,NaN,えむえーえす,C,-1,B,-1,C,-1,...,0.0,C,0.0,C,NaN,NaN,22.0,0-5%,MANBYO;JST,S


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     9999 non-null   object 
 1   出現形                    9999 non-null   object 
 2   出現形_flag               3 non-null      object 
 3   出現形よみ                  9999 non-null   object 
 4   出現形よみ_flag             9999 non-null   object 
 5   正規形                    6627 non-null   object 
 6   正規形_flag               6444 non-null   object 
 7   正規形よみ                  6363 non-null   object 
 8   正規形よみ_flag             3465 non-null   object 
 9   TREE                   9987 non-null   object 
 10  TREE_flag              9999 non-null   object 
 11  出現形EN                  9988 non-null   object 
 12  出現形EN_flag             9988 non-null   object 
 13  出現形ZH                  9988 non-null   object 
 14  出現形ZH_flag             9988 non-null   object 
 15  REL:

In [16]:
df["temp_flag"]

0       S
1       S
2       S
3       S
4       S
       ..
9994    S
9995    S
9996    S
9997    S
9998    S
Name: temp_flag, Length: 9999, dtype: object

In [17]:
df[["出現形", "正規形", "正規形よみ","TO:ICD-10", "出現形よみ_flag", "temp_flag"]].iloc[8992:9002,:]

,出現形,正規形,正規形よみ,TO:ICD-10,出現形よみ_flag,temp_flag
8992,心臓腫よう,NaN,NaN,I254,D,D
8993,残存腫,NaN,NaN,D481,D,D
8994,化膿,NaN,NaN,L080,D,D
8995,呼吸異常,異常呼吸,いじょうこきゅう,R068,D,D
8996,巻き爪,NaN,NaN,L604,D,D
8997,食欲減退,食欲不振,しょくよくふしん,R630,C,D
8998,ステロイド糖尿病,ステロイド糖尿病,すてろいどとうにょうびょう,E13,S,S
8999,Ｓｗｅｅｔ病,スイート病,すいーとびょう,L982,A,S
9000,心不全合併,心不全,しんふぜん,I509,C,S
9001,腹部大動脈りゅう破裂,腹部大動脈瘤破裂,ふくぶだいどうみゃくりゅうはれつ,I713,C,S


In [18]:
df[["出現形", "正規形", "正規形よみ","TO:ICD-10", "出現形よみ_flag", "temp_flag"]].iloc[:5,:]

,出現形,正規形,正規形よみ,TO:ICD-10,出現形よみ_flag,temp_flag
0,副作用,-1,-1,-1,C,S
1,合併症,-1,-1,-1,B,S
2,糖尿病,糖尿病,とうにょうびょう,E14,S,S
3,再発,-1,-1,-1,C,S
4,癌,癌,がん,C809,S,S


In [19]:
df[df["temp_flag"]=="S"]["出現形"].str.len().describe()

count    2000.000000
mean        4.778000
std         2.770298
min         1.000000
25%         3.000000
50%         4.000000
75%         6.000000
max        25.000000
Name: 出現形, dtype: float64

In [20]:
df[df["temp_flag"]=="S"]["正規形"].str.len().describe()

count    2000.000000
mean        4.636000
std         7.024042
min         1.000000
25%         2.000000
50%         4.000000
75%         5.000000
max       270.000000
Name: 正規形, dtype: float64

In [21]:
ranks = "SABCDE"
rank = "S"
input_column="出現形"
target_column="正規形"
reliability_column="temp_flag"
ix_train = ranks.index(rank) + 1  # => 1 + 1 = 2
ranks_train = ranks[:ix_train]    # => "SA"
str_ranks_train = r"|".join(r for r in ranks_train)  # => "S|A"
mask_train = df[reliability_column].fillna("E").str.contains(str_ranks_train)
df_train = df[mask_train][[input_column, target_column, reliability_column]]
df_test = df[~mask_train][[input_column, target_column, reliability_column]]  # ~mask_train == not mask_train


In [22]:
df_train.iloc[995:1002,:]

,出現形,正規形,temp_flag
995,横紋筋融解症,横紋筋融解,S
996,行動変容,[ERR],S
997,薬剤,-1,S
998,じ,[ERR],S
8998,ステロイド糖尿病,ステロイド糖尿病,S
8999,Ｓｗｅｅｔ病,スイート病,S
9000,心不全合併,心不全,S


In [23]:
# 事前学習済みモデル
# MODEL_NAME = "megagonlabs/t5-base-japanese-web"
MODEL_NAME = "retrieva-jp/t5-base-long"

In [24]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, is_fast=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
# Trainerの初期化とトレーニング
trainer = Trainer(model_name=MODEL_NAME,tokenizer = tokenizer, train_data = df_train.head(100),
                  input_column=input_column, target_column=target_column, batch_size=128,
                  token_max_length_src=16, token_max_length_tgt=16, epochs=20, device = device)
best_model, best_tokenizer, timestamp, stop_epoch = trainer.train()

 25%|██▌       | 5/20 [00:04<00:11,  1.25it/s]

Epoch: 5, Train Loss: 46.08292770385742
Val Accuracy: 0.0, Val Loss: inf
Targets: ['敗血症', '不安障害', '悪性リンパ腫', 'リンパ節転移', '損傷', '虚血性心疾患', '発熱', '統合失調症', '出血', '-1']
Predicts: ['高血圧症の薬 - 日本最大級感染症情報センター', '不安', '悪性リンパ腫(HLA)胸腺切除術', 'リンパ節切除術とは? 原因と症状を徹底解説', '|の修理・補修方法と相場価格を比較', '検査 参考情報に関するご意見、質問などがございましたらお気軽に', 'の通販はWowma!(ワウマ)', '精神分裂病患者のブログです。、自殺しました。', '| 固定リンク', 'とは、のことです。 トップページ > 副作用をなくす']


 50%|█████     | 10/20 [00:07<00:06,  1.46it/s]

Epoch: 10, Train Loss: 44.890018463134766
Val Accuracy: 0.0, Val Loss: inf
Targets: ['敗血症', '不安障害', '悪性リンパ腫', 'リンパ節転移', '損傷', '虚血性心疾患', '発熱', '統合失調症', '出血', '-1']
Predicts: ['- 高血圧の薬- | 教えて!Goo,知恵', '不安', '| 悪性リンパ腫|腫瘍内科胸腺肥大', 'リンパ節転移(黄体細胞腫)の手術方法と', 'の修理は、を補修する必要があります。', '冠動脈疾患 参考情報に関するご意見、お問い合せ内科', '', '精神分裂病患者のブログです。| HOME | 2ちゃんねる', '| 固定リンク', 'とは、のことです。']


 75%|███████▌  | 15/20 [00:11<00:03,  1.45it/s]

Epoch: 15, Train Loss: 43.73502731323242
Val Accuracy: 0.0, Val Loss: inf
Targets: ['敗血症', '不安障害', '悪性リンパ腫', 'リンパ節転移', '損傷', '虚血性心疾患', '発熱', '統合失調症', '出血', '-1']
Predicts: ['', '不安', '悪性リンパ腫乳癌の再発と治療 |', 'リンパ節転移 | 癌治療のタキソサイシン', '|の修理・補修方法と費用は、', '血液疾患 参考情報に関するご注意虚血性心疾患', 'Copyright All Rights Reserved.', '精神分裂病の憂鬱( ́・ω・)', '| 固定リンク', 'とは、のことです。']


100%|██████████| 20/20 [00:14<00:00,  1.36it/s]

Epoch: 20, Train Loss: 42.65061569213867
Val Accuracy: 0.0, Val Loss: inf
Targets: ['敗血症', '不安障害', '悪性リンパ腫', 'リンパ節転移', '損傷', '虚血性心疾患', '発熱', '統合失調症', '出血', '-1']
Predicts: ['敗血症性大腸菌(C型肝炎)', '不安', '悪性リンパ腫乳腺癌の手術と治療', 'リンパ節転移 [癌]の症状と治療について |', ',', '参考情報に「虚血性心疾患」と記載があります。', '', '精神分裂病', '| 固定リンク', 'とは、のことです。']


In [38]:
# best_model = T5ForConditionalGeneration.from_pretrained(
#     "./models/retrieva-jp/t5-base-long/appearance_toappearance_yomi/2024-02-16T19:15:48/model"
# )
# best_tokenizer = T5Tokenizer.from_pretrained(
#     "./models/retrieva-jp/t5-base-long/appearance_toappearance_yomi/2024-02-16T19:15:48/tokenizer"
# )
# timestamp = "2024-02-16T19:15:48"

In [39]:
# テキスト生成の準備
text_generator = GenerateText(best_model, best_tokenizer,df_test, input_column, target_column, timestamp,
                              batch_size=128, token_max_length_src=16, token_max_length_tgt=16, device=device)

# テキスト生成
generated_df = text_generator.generate_text()

  1%|          | 35/3842 [00:55<1:38:32,  1.55s/it]

100%|██████████| 3842/3842 [2:20:22<00:00,  2.19s/it]  


OSError: Cannot save file into a non-existent directory: 'data/outputs'

In [ ]:
generated_df

,appearance,appearance_yomi,appearance_yomi_generated
0,副作用,ふくさよう,ようこう
1,合併症,がっぺいしょう,けっしょう
2,再発,さいはつ,さいはつ
3,高血圧,こうけつあつ,はくしょう
4,痛み,いたみ,しっせつ
...,...,...,...
9995,奇脈,きみゃく,きみゃく
9996,再発大腸癌,さいはつだいちょうがん,さいせいだいちょうがん
9997,二次性アミロイドーシス,にじせいあみろいどーしす,じせいあみろいどー
9998,慢性閉塞性動脈硬化症,まんせいへいそくせいどうみゃくこうかしょう,まんせいへいそくせいどうみゃく
